# KRX 데이터 조회 - 전일급등주
- description: 가즈아
- layout: post
- categories: [경제]
- toc: true

In [1]:
#hide
!pip install pandas pandasql xlrd

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=f77bcfeeb6b740ae058985aa64551d41b4835b662f12a532e16f3c52adf8a82b
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [2]:
# hide
import requests
import pandas as pd
from pandas.tseries.offsets import BDay
from io import StringIO
from io import BytesIO
import json
import datetime

왜 이렇게 만들었는지 모르겠는데 KRX는 한번 꼬아서 다운로드 받는다. 이러면 내가 못할 줄 알고?

In [3]:
def downloadKrxData(cd):
    resp = requests.post(
        'http://file.krx.co.kr/download.jspx', 
        headers={
            'User-Agent': 'Mozilla/5.0', 
            'Referer': 'http://marketdata.krx.co.kr/',
        }, 
        params={'code': cd},
    )
    tp = resp.headers['Content-Disposition'].split('.')[1]
    if tp == 'csv':
        return pd.read_csv(StringIO(resp.text))
    if tp == 'xls':
        return pd.read_excel(BytesIO(resp.content))

## KRX 휴장일을 반영한 전영업일 
- KRX 01023 휴장일 http://marketdata.krx.co.kr/mdi#document=01100305

In [4]:
def closingDays(year):
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'xls',
            'url': 'MKD/01/0110/01100305/mkd01100305_01',
            'search_bas_yy': year,
            'gridTp': 'KRX',
            'pagePath': '/contents/MKD/01/0110/01100305/MKD01100305.jsp',
        },
    )
    return downloadKrxData(resp.text)['일자 및 요일']

def getPreviousBizDay(today):
    pday = datetime.datetime.strptime(today, '%Y%m%d')
    yyyy = pday.year
    cds = closingDays(yyyy)

    while True:
        pday = pday - BDay(1)
        if yyyy != pday.year:
            yyyy = pday.year
            cds = pd.concat([cds, closingDays(yyyy)])
        if pday.strftime("%Y-%m-%d") not in cds.values:
            return pday.strftime("%Y%m%d")

## 전일 등락
- KRX 80037 전체종목 등락률 http://marketdata.krx.co.kr/mdi#document=13020102

In [5]:
def soaredStocks(dt):
    print(dt)
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'csv',
            'url': 'MKD/13/1302/13020102/mkd13020102',
            'ind_tp': 'ALL',
            'period_strt_dd': dt,
            'period_end_dd': dt,
            'pagePath': '/contents/MKD/13/1302/13020102/MKD13020102.jsp',
        },
    )
    return downloadKrxData(resp.text)

In [6]:
# hide
pd.set_option('display.max.colwidth', 150)

## 전 영업일 15% 이상 오른 종목

In [7]:
dt = getPreviousBizDay(datetime.datetime.now().strftime('%Y%m%d'))
# dt = '20201026'
df_soarings = soaredStocks(dt)

20201116


In [8]:
# hide
df_15 = df_soarings[df_soarings.등락률 >= 15].sort_values('등락률', ascending=False)
df_15['네이버주식'] = 'https://finance.naver.com/item/main.nhn?code=' + df_15['종목코드']
df_15

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
2406,323990,박셀바이오,"63,000","81,900","18,900",30.00,"1,003,594","78,865,760,800",https://finance.naver.com/item/main.nhn?code=323990
2241,267850,아시아나IDT,"24,000","31,200","7,200",30.00,"4,731,001","139,087,744,150",https://finance.naver.com/item/main.nhn?code=267850
1747,130500,GH신소재,"6,200","8,060","1,860",30.00,"12,460,302","91,535,025,950",https://finance.naver.com/item/main.nhn?code=130500
179,002995,금호산업우,"40,850","53,100","12,250",29.99,"40,426","2,144,858,000",https://finance.naver.com/item/main.nhn?code=002995
208,003495,대한항공우,"27,900","36,250","8,350",29.93,"1,265,113","42,564,569,050",https://finance.naver.com/item/main.nhn?code=003495
2329,298690,에어부산,"3,845","4,995","1,150",29.91,"24,371,849","114,051,697,690",https://finance.naver.com/item/main.nhn?code=298690
2346,304840,피플바이오,"43,350","56,300","12,950",29.87,"4,542,398","240,347,601,050",https://finance.naver.com/item/main.nhn?code=304840
656,020560,아시아나항공,"4,290","5,570","1,280",29.84,"141,613,627","743,745,121,745",https://finance.naver.com/item/main.nhn?code=020560
178,002990,금호산업,"7,910","10,250","2,340",29.58,"4,483,195","43,274,086,290",https://finance.naver.com/item/main.nhn?code=002990
1490,090710,휴림로봇,"1,620","2,030",410,25.31,"57,413,835","111,381,348,865",https://finance.naver.com/item/main.nhn?code=090710


In [9]:
import pandasql as ps
ps.sqldf('''
    select a.*,
           'https://finance.naver.com/item/main.nhn?code=' || a.종목코드 as '네이버주식'
      from df_soarings a
     where 등락률 >= 15
  order by 등락률 desc
''')

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
0,130500,GH신소재,"6,200","8,060","1,860",30.00,"12,460,302","91,535,025,950",https://finance.naver.com/item/main.nhn?code=130500
1,267850,아시아나IDT,"24,000","31,200","7,200",30.00,"4,731,001","139,087,744,150",https://finance.naver.com/item/main.nhn?code=267850
2,323990,박셀바이오,"63,000","81,900","18,900",30.00,"1,003,594","78,865,760,800",https://finance.naver.com/item/main.nhn?code=323990
3,002995,금호산업우,"40,850","53,100","12,250",29.99,"40,426","2,144,858,000",https://finance.naver.com/item/main.nhn?code=002995
4,003495,대한항공우,"27,900","36,250","8,350",29.93,"1,265,113","42,564,569,050",https://finance.naver.com/item/main.nhn?code=003495
5,298690,에어부산,"3,845","4,995","1,150",29.91,"24,371,849","114,051,697,690",https://finance.naver.com/item/main.nhn?code=298690
6,304840,피플바이오,"43,350","56,300","12,950",29.87,"4,542,398","240,347,601,050",https://finance.naver.com/item/main.nhn?code=304840
7,020560,아시아나항공,"4,290","5,570","1,280",29.84,"141,613,627","743,745,121,745",https://finance.naver.com/item/main.nhn?code=020560
8,002990,금호산업,"7,910","10,250","2,340",29.58,"4,483,195","43,274,086,290",https://finance.naver.com/item/main.nhn?code=002990
9,090710,휴림로봇,"1,620","2,030",410,25.31,"57,413,835","111,381,348,865",https://finance.naver.com/item/main.nhn?code=090710
